In [1]:
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, precision_score
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

In [12]:
# shutil.rmtree('./model_checkpoints', ignore_errors=True)

X = df_combo
y = labels

input_shape = len(X.columns)

random_state = 42

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.235, random_state=random_state, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.5, random_state=random_state, stratify=y_val)

scaler = StandardScaler()
X_train_scaled, X_val_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_val), scaler.transform(X_test)

zero_weight = (np.sum(y_train==1)/np.sum(y_train==0))*1.5

# Create a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, activation='relu',
              input_shape=(input_shape,),
              kernel_regularizer=tf.keras.regularizers.L1(0.001),
              activity_regularizer=tf.keras.regularizers.L2(0.0015),
              kernel_initializer=tf.keras.initializers.GlorotUniform()
              ),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='sigmoid',
              kernel_initializer=tf.keras.initializers.GlorotUniform(),
                          ),
])

# Convert target labels to one-hot encoding
y_train_categorized, y_val_categorized = to_categorical(y_train, num_classes=2), to_categorical(y_val, num_classes=2)

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# Define a callback to save checkpoints
checkpoint_filepath = "./model_checkpoints/checkpoint_{epoch:1d}.h5"

# Save a model checkpoint every 2 epochs
model_save = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    period=2,  # Save every 2 epochs
)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.AUC()])
epochs = 20
# Train the model
history = model.fit(X_train_scaled, y_train_categorized, epochs=epochs, batch_size=32, class_weight={0:zero_weight, 1:1}, validation_data=(X_val_scaled, y_val_categorized), callbacks=[early_stopping, model_save])

Epoch 1/20
122/122 [==============================] - 2s 8ms/step - loss: 7.9672 - auc_3: 0.8022 - val_loss: 5.3544 - val_auc_3: 0.8922
Epoch 2/20
122/122 [==============================] - 1s 5ms/step - loss: 3.3232 - auc_3: 0.9399 - val_loss: 2.1183 - val_auc_3: 0.9298
Epoch 3/20
122/122 [==============================] - 1s 5ms/step - loss: 1.2675 - auc_3: 0.9616 - val_loss: 1.0049 - val_auc_3: 0.9345
Epoch 4/20
122/122 [==============================] - 1s 6ms/step - loss: 0.6186 - auc_3: 0.9668 - val_loss: 0.6675 - val_auc_3: 0.9351
Epoch 5/20
122/122 [==============================] - 1s 5ms/step - loss: 0.4379 - auc_3: 0.9605 - val_loss: 0.5729 - val_auc_3: 0.9465
Epoch 6/20
122/122 [==============================] - 1s 5ms/step - loss: 0.4003 - auc_3: 0.9602 - val_loss: 0.5197 - val_auc_3: 0.9545
Epoch 7/20
122/122 [==============================] - 1s 5ms/step - loss: 0.3840 - auc_3: 0.9613 - val_loss: 0.5218 - val_auc_3: 0.9507
Epoch 8/20
122/122 [============================

In [5]:
def get_scores(actual, pred):
    fpr, tpr, thresholds = roc_curve(actual,pred)
    auc_score = roc_auc_score(actual, pred)

    best_threshold = (tpr+(1-fpr)/2)
    best_threshold = thresholds[np.argmax(best_threshold)]
    pred = (pred >= best_threshold).astype(int)

    precision = precision_score(actual, pred)
    recall = recall_score(actual, pred)
    f1 = f1_score(actual, pred)

    return {"Thresh":best_threshold, "Precision":precision, "Recall":recall, "F1_Score":f1, "AUC":auc_score}

In [13]:
#Testing on test set
scores = []

try:
  # Loop through all model checkpoint
  # Start = 0
  # End = Number of epochs / epoch_save_period + 1
  for i in range(1,int(epochs/2)+1):
    path = f'./model_checkpoints/checkpoint_{i*2}.h5'

    model.load_weights(path)
    y_pred = model.predict(X_test_scaled)[:,1]
    score = get_scores(y_test, y_pred)
    score['checkpoint'] = i*2
    scores.append(score)
except:
  None

19/19 [==============================] - 0s 2ms/step


In [14]:
#Display scores of each checkpoint
scores = pd.DataFrame(scores).sort_values(by='AUC', ascending=False)
scores.head()

,Thresh,Precision,Recall,F1_Score,AUC,checkpoint
8,0.299788,0.611650,0.969231,0.750000,0.958265,18
7,0.296486,0.617647,0.969231,0.754491,0.956809,16
9,0.390810,0.661376,0.961538,0.783699,0.954157,20
6,0.336216,0.631313,0.961538,0.762195,0.953993,14
5,0.295033,0.593458,0.976923,0.738372,0.952619,12
